In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats


# 1. Convert cell counts to relative frequencies

In [7]:
def process_cell_counts(input_file, output_file):
    df = pd.read_csv(input_file)
    populations = ['b_cell', 'cd8_t_cell', 'cd4_t_cell', 'nk_cell', 'monocyte']
    df['total_count'] = df[populations].sum(axis=1)
    melted_df = df.melt(
        id_vars=['sample', 'total_count'],
        value_vars=populations,
        var_name='population',
        value_name='count'
    )
    melted_df['percentage'] = (melted_df['count'] / melted_df['total_count']) * 100
    result_df = melted_df[['sample', 'total_count', 'population', 'count', 'percentage']]
    result_df.to_csv(output_file, index=False)
    return result_df
relative_freq_df = process_cell_counts('cell-count.csv', 'cell-count-relative-freq.csv')

# 2. Analysis of tr1 treatment responders vs non-responders

In [15]:
def analyze_tr1_response(df):
    tr1_df = df[(df['treatment'] == 'tr1') & (df['sample_type'] == 'PBMC')].copy()
    tr1_melanoma_df = tr1_df[tr1_df['condition'] == 'melanoma'].copy()
    merged_df = pd.merge(
        tr1_melanoma_df,
        relative_freq_df,
        on='sample'
    )
    
    #boxplots
    populations = ['b_cell', 'cd8_t_cell', 'cd4_t_cell', 'nk_cell', 'monocyte']
    for pop in populations:
        plt.figure(figsize=(8, 6))
        sns.boxplot(
            x='response',
            y='percentage',
            data=merged_df[merged_df['population'] == pop]
        )
        plt.title(f'{pop} percentage by response')
        plt.ylabel('Percentage (%)')
        plt.xlabel('Response (y=responder, n=non-responder)')
        plt.savefig(f'{pop}_response_comparison.png')
        plt.close()
    
    # statistical tests
    significant_pops = []
    for pop in populations:
        responders = merged_df[(merged_df['population'] == pop) & (merged_df['response'] == 'y')]['percentage']
        non_responders = merged_df[(merged_df['population'] == pop) & (merged_df['response'] == 'n')]['percentage']
        if len(responders) > 1 and len(non_responders) > 1:
            t_stat, p_val = stats.ttest_ind(responders, non_responders)
            
            if p_val < 0.05:
                significant_pops.append({
                    'population': pop,
                    'p_value': p_val,
                    'mean_responder': np.mean(responders),
                    'mean_non_responder': np.mean(non_responders),
                    't_statistic': t_stat
                })
    
    return pd.DataFrame(significant_pops)
significant_pops_df = analyze_tr1_response(pd.read_csv('cell-count.csv'))

In [19]:
database_schema = """
# Database Schema Design

1. **Projects Table**
   - project_id (PK): Unique identifier for each project
   - project_name: Name of the project
   - description: Description of the project
   - start_date: When the project started
   - end_date: When the project ended

2. **Subjects Table**
   - subject_id (PK): Unique identifier for each subject (sbj1, sbj2, etc.)
   - project_id (FK): Reference to Projects table
   - age: Age of the subject
   - sex: Sex of the subject (M/F)
   - condition: Medical condition (melanoma, lung, healthy, etc.)

3. **Treatments Table**
   - treatment_id (PK): Unique identifier for each treatment
   - treatment_name: Name of treatment (tr1, tr2, etc.)
   - description: Description of the treatment

4. **Samples Table**
   - sample_id (PK): Unique identifier for each sample (s1, s2, etc.)
   - subject_id (FK): Reference to Subjects table
   - sample_type: Type of sample (PBMC, tumor, etc.)
   - collection_date: Date when sample was collected
   - time_from_treatment_start: Days from treatment start

5. **Cell_Populations Table**
   - population_id (PK): Unique identifier for each cell population
   - population_name: Name of population (b_cell, cd8_t_cell, etc.)
   - description: Description of the cell population

6. **Cell_Counts Table**
   - count_id (PK): Unique identifier
   - sample_id (FK): Reference to Samples table
   - population_id (FK): Reference to Cell_Populations table
   - count: Absolute cell count
   - percentage: Relative percentage

7. **Treatment_Response Table**
   - response_id (PK): Unique identifier
   - subject_id (FK): Reference to Subjects table
   - treatment_id (FK): Reference to Treatments table
   - response: Response (y/n)
   - response_date: Date when response was assessed

Advantages of using a database:
1. Data integrity and consistency
2. Efficient querying of large datasets
3. Better scalability for hundreds of projects/thousands of samples
4. Ability to establish relationships between entities
5. Support for concurrent access
6. Advanced analytics capabilities
"""

# Example Queries
queries = """
-- Query 3: Summarize number of subjects for each condition
SELECT condition, COUNT(DISTINCT subject_id) as subject_count
FROM Subjects
GROUP BY condition
ORDER BY subject_count DESC;

-- Query 4: Melanoma PBMC samples at baseline with tr1 treatment
SELECT s.sample_id, s.subject_id, subj.condition, s.sample_type, s.time_from_treatment_start
FROM Samples s
JOIN Subjects subj ON s.subject_id = subj.subject_id
JOIN Treatment_Response tr ON subj.subject_id = tr.subject_id
WHERE subj.condition = 'melanoma'
AND s.sample_type = 'PBMC'
AND s.time_from_treatment_start = 0
AND tr.treatment_id = (SELECT treatment_id FROM Treatments WHERE treatment_name = 'tr1')
AND subj.condition = 'bladder cancer';  -- As requested

-- Query 5a: Samples from each project
SELECT p.project_name, COUNT(s.sample_id) as sample_count
FROM Samples s
JOIN Subjects subj ON s.subject_id = subj.subject_id
JOIN Projects p ON subj.project_id = p.project_id
WHERE subj.condition = 'melanoma'
AND s.sample_type = 'PBMC'
AND s.time_from_treatment_start = 0
GROUP BY p.project_name;

-- Query 5b: Responders/non-responders
SELECT tr.response, COUNT(DISTINCT s.sample_id) as sample_count
FROM Samples s
JOIN Subjects subj ON s.subject_id = subj.subject_id
JOIN Treatment_Response tr ON subj.subject_id = tr.subject_id
WHERE subj.condition = 'melanoma'
AND s.sample_type = 'PBMC'
AND s.time_from_treatment_start = 0
GROUP BY tr.response;

-- Query 5c: Males and females
SELECT subj.sex, COUNT(DISTINCT s.sample_id) as sample_count
FROM Samples s
JOIN Subjects subj ON s.subject_id = subj.subject_id
WHERE subj.condition = 'melanoma'
AND s.sample_type = 'PBMC'
AND s.time_from_treatment_start = 0
GROUP BY subj.sex;
"""

# Save the significant populations to a file
significant_pops_df.to_csv('significant_populations.csv', index=False)

# Print results
print("Relative frequencies calculated and saved to cell-count-relative-freq.csv")
print("\nSignificantly different populations between responders and non-responders:")
print(significant_pops_df)
print("\nDatabase schema design:")
print(database_schema)
print("\nExample queries:")
print(queries)


Relative frequencies calculated and saved to cell-count-relative-freq.csv

Significantly different populations between responders and non-responders:
   population   p_value  mean_responder  mean_non_responder  t_statistic
0  cd4_t_cell  0.008351       36.333333           26.333333     3.634219
1    monocyte  0.033508        8.000000           22.666667    -2.638267

Database schema design:

# Database Schema Design

1. **Projects Table**
   - project_id (PK): Unique identifier for each project
   - project_name: Name of the project
   - description: Description of the project
   - start_date: When the project started
   - end_date: When the project ended

2. **Subjects Table**
   - subject_id (PK): Unique identifier for each subject (sbj1, sbj2, etc.)
   - project_id (FK): Reference to Projects table
   - age: Age of the subject
   - sex: Sex of the subject (M/F)
   - condition: Medical condition (melanoma, lung, healthy, etc.)

3. **Treatments Table**
   - treatment_id (PK): Unique i